In [44]:
import numpy as np
import pandas as pd
import sys
import pymysql
import random
from pytz import timezone
from datetime import datetime
from sklearn.feature_extraction import text
from sklearn.metrics.pairwise import cosine_similarity

In [2]:
lmr = pymysql.connect(
    user = 'lmr',
    passwd = 'lmr',
    host = '52.78.72.175',
    db = 'lmr_db'
)
cur = lmr.cursor()

user_id = 72로 설정

In [3]:
sql_user = "SELECT * FROM lmr_db.user"
sql_user_allergy = "SELECT * FROM lmr_db.user_allergy WHERE user_id = 72"
sql_user_prefer = "SELECT * FROM lmr_db.preferred_menu WHERE user_id = 72"
sql_user_log = "SELECT * FROM lmr_db.menu_recommend_log WHERE user_id = 72"

In [4]:
cur.execute(sql_user)
rows = cur.fetchall()
user = pd.DataFrame(rows)
user.columns = ['id', 'password', 'last_login', 'is_superuser', 'email', 'username', 'nickname', 'gender', 'age', 'introduction', 'is_staff', 'is_active', 'date_joined']
user.tail()

,id,password,last_login,is_superuser,email,username,nickname,gender,age,introduction,is_staff,is_active,date_joined
1,71,pbkdf2_sha256$260000$ODctgjG703oOAWUqen6Xts$rY...,2023-05-25 13:13:40.434396,0,hye@h.com,None,박혜인,여자,2000.0,한 줄 소개,0,1,2023-05-25 07:09:14.049895
2,72,pbkdf2_sha256$260000$TiVLfXScEig4iMO4moVENn$gp...,2023-05-25 09:10:20.854067,0,yun@y.com,None,이윤하,여자,2000.0,한 줄 소개,0,1,2023-05-25 07:15:32.445176
3,73,pbkdf2_sha256$260000$UapFjVqnM32ImwsGA09GxM$5q...,2023-05-25 08:46:46.502816,0,jung@j.com,None,정의헌,남자,2000.0,한 줄 소개,0,1,2023-05-25 07:58:53.285658
4,76,pbkdf2_sha256$260000$6EK22PIFJ4PsxJhm90khkx$gb...,2023-05-25 09:07:13.171952,0,seo@s.com,None,서제원,남자,1999.0,한 줄 소개,0,1,2023-05-25 08:52:03.492690
5,77,pbkdf2_sha256$260000$nwfnRLrVFuCzvTznuKpx0d$W6...,2023-05-25 09:31:30.880788,0,b@b.com,None,밥푸리,남자,2000.0,한 줄 소개,0,1,2023-05-25 09:31:01.856944


In [5]:
cur.execute(sql_user_allergy)
rows = cur.fetchall()
user_allergy = pd.DataFrame(rows)
user_allergy.columns = ['id', '달걀', '우유', '밀', '콩', '땅콩', '생선', '고기', '조개', '갑각류', 'user_id']
user_allergy.tail()

,id,달걀,우유,밀,콩,땅콩,생선,고기,조개,갑각류,user_id
0,64,0,0,1,0,0,0,0,0,0,72


In [6]:
cur.execute(sql_user_prefer)
rows = cur.fetchall()
user_prefer = pd.DataFrame(rows)
user_prefer.columns = ['id', 'preference', 'menu_id', 'user_id']
user_prefer

,id,preference,menu_id,user_id
0,478,1,66,72
1,479,1,189,72
2,480,1,289,72
3,481,1,213,72
4,482,1,39,72
5,483,-1,259,72
6,484,-1,316,72
7,485,1,328,72
8,486,1,94,72
9,487,-1,183,72


In [7]:
cur.execute(sql_user_log)
rows = cur.fetchall()
user_log = pd.DataFrame(rows)
user_log.columns = ['id', 'datetime', 'menu_id', 'user_id']
user_log.tail()

,id,datetime,menu_id,user_id
10,133,2023-05-25 07:20:24.827131,82,72
11,134,2023-05-25 07:20:26.634830,237,72
12,135,2023-05-25 07:20:28.795654,212,72
13,136,2023-05-25 07:20:31.450544,314,72
14,137,2023-05-25 07:20:33.571676,61,72


In [8]:
sql_menu = "SELECT * FROM lmr_db.menu"
sql_nutrition = "SELECT * FROM lmr_db.nutrition"
sql_restaurant = "SELECT * FROM lmr_db.restaurant"

In [9]:
cur.execute(sql_menu)
rows = cur.fetchall()
menu = pd.DataFrame(rows)
menu.columns = ['id', 'category', 'name', 'price', 'emotion', 'weather', 'image', 'restaurant_id']
menu.head()

,id,category,name,price,emotion,weather,image,restaurant_id
0,1,국밥,양평해장국,9000,"배고픔,기쁨,피곤,스트레스","맑음,구름,비,눈",menu/Restaurant object (1)/양평해장국.jpg,1
1,2,국밥,양평해장국(특),10000,"배고픔,기쁨,피곤,스트레스","맑음,구름,비,눈",menu/Restaurant object (1)/양평해장국특.jpg,1
2,3,국밥,뼈다귀해장국,10000,"배고픔,기쁨,피곤,스트레스","맑음,구름,비,눈",menu/Restaurant object (1)/뼈다귀해장국.jpg,1
3,4,국밥,얼큰순대국,9000,"배고픔,기쁨,피곤,스트레스","맑음,구름,비,눈",menu/Restaurant object (1)/얼큰순대국.jpg,1
4,5,국밥,내장탕,13000,"배고픔,기쁨,피곤,스트레스","맑음,구름,비,눈",menu/Restaurant object (1)/내장탕.jpg,1


In [10]:
cur.execute(sql_nutrition)
rows = cur.fetchall()
nutrient_menu = pd.DataFrame(rows)
nutrient_menu.columns = ['id', 'name', 'gram', 'calorie', 'carbohydrate', 'protein', 'fat', 'saturatedfat', 'unsaturatedfat', 'cholesterol', 'sodium', 'potash', 'ingredient', 'allergy', 'menu_id']
nutrient_menu = nutrient_menu.sort_values(by=['menu_id'])
nutrient_menu.head()

,id,name,gram,calorie,carbohydrate,protein,fat,saturatedfat,unsaturatedfat,cholesterol,sodium,potash,ingredient,allergy,menu_id
1,2,양평해장국,500.0,662.0,16.47,83.210,28.560,9.360,12.7370,222.0,4199.0,1363.0,"마늘, 생강, 대파",None,1
2,3,양평해장국(특),600.0,794.4,197.64,99.852,34.272,11.232,15.2844,266.0,5039.0,1636.0,"마늘, 생강, 대파",None,2
0,1,뼈해장국,300.0,315.0,7.10,39.940,13.540,4.480,6.0900,106.0,2420.0,626.0,"돼지고기, 우거지, 대파, 마늘",고기,3
3,4,얼큰순대국,300.0,328.0,33.78,23.250,10.350,3.615,3.1450,439.0,1953.0,485.0,"순대, 사골, 마늘, 대파, 쌀, 돼지고기",고기,4
5,6,내장탕,500.0,403.0,33.04,29.670,16.860,5.373,5.7030,356.0,1107.0,1187.0,"내장, 파, 된장, 마늘",콩,5


In [11]:
cur.execute(sql_restaurant)
rows = cur.fetchall()
restaurant = pd.DataFrame(rows)
restaurant.columns = ['id', 'name', 'address', 'business_hours', 'phone_number', 'category_name', 'image', 'latitude', 'longtitude']
restaurant.head()

,id,name,address,business_hours,phone_number,category_name,image,latitude,longtitude
0,1,일품양평해장국 인천대점,인천 연수구 송도동 13-58,10:00 ~ 22:00,032-214-0359,음식점 > 한식 > 해장국 > 일품양평해장국,restaurant/일품양평해장국로고.jpg,37.374014,126.636376
1,2,스노우폭스 송도점,인천 연수구 송도동 13-58,07:30 ~ 20:00,032-710-6464,"음식점 > 일식 > 초밥,롤",restaurant/스노우폭스_로고.jpg,37.373991,126.636597
2,3,오백국수 송도인천대점,인천 연수구 송도동 13-58,11:00 ~ 20:00,032-859-7788,음식점 > 한식 > 국수,restaurant/오백국수로고.jpg,37.373991,126.636597
3,4,맘스터치 인천대제3기숙사점,인천 연수구 송도동 12-1,11:00 ~ 20:00,032-858-3211,음식점 > 패스트푸드 > 맘스터치,restaurant/맘스터치로고.jpg,126.629720,126.629720
4,5,샐러디 인천대점,인천 연수구 송도동 12-1,10:00 ~ 20:00,070-4155-2222,음식점 > 샐러드 > 샐러디,restaurant/샐러디.jpg,37.372859,126.631691


In [12]:
### 메인 기능 : 사용자 선호 메뉴를 중심으로 메뉴 추천 ###

# 1) 사용자가 좋아하는 메뉴 다 가져오기
like_menu_list = list() ; dislike_menu_list = list()

for i in user_prefer.itertuples():
    
    # 좋아하는 메뉴
    if i.preference == 1:
        like_menu_list.append(i.menu_id)

    # 싫어하는 메뉴
    elif i.preference == -1:
        dislike_menu_list.append(i.menu_id)

print('좋아하는 메뉴 :', like_menu_list)
print('싫어하는 메뉴 :', dislike_menu_list)

좋아하는 메뉴 : [66, 189, 289, 213, 39, 328, 94, 193, 150, 230, 112, 156, 366, 335, 237, 229, 248, 192]
싫어하는 메뉴 : [259, 316, 183, 68, 223, 334, 268, 258, 317]


In [96]:
# 2) 특징 데이터를 사용하여 텍스트 특징 추출

# 메뉴 특징 데이터 생성 및 menu에 컬럼 추가
# nutrient_menu 데이터의 ingredient 피처를 menu_id 기준으로 정렬
nutrient_menu = nutrient_menu.sort_values(by=['menu_id'])
tmp_df = nutrient_menu['ingredient']
tmp_df = tmp_df.reset_index(drop=True)

# 메뉴 데이터 특징 피처 만들어서 저장
menu['feature'] = menu['category'] + " " + menu['name'] + " " + menu['weather'] + " " + menu['emotion'] + " " + tmp_df
menu_feature = menu['feature'].tolist()

# TF-IDF Vectorizer Object 구현
tfidf = text.TfidfVectorizer()
tfidf_matrix = tfidf.fit_transform(menu_feature)

# 코사인 유사도 계산
similarity = cosine_similarity(tfidf_matrix)

# 중복된 추천 제외하고, 특징 데이터 사용
indices = pd.Series(menu.index, index=menu.feature).drop_duplicates()

# 사용자 선호 메뉴와 유사한 메뉴 추천 (좋아하는 메뉴만큼 for문 진행)
rcm_menu = list()
rcm_similarity = list()

for i in like_menu_list:
    for j in range(len(menu)):
        if i == menu.iloc[j]['id']:
            feature = menu.iloc[j]['feature']

    index = indices[feature]

    similarity_scores = list(enumerate(similarity[index]))
    similarity_scores = sorted(similarity_scores, key=lambda x: x[1], reverse=True)
    similarity_scores = similarity_scores[1:11]        # 결과는 메뉴 아이디, 유사도 (10개씩 추출)
    menu_indices = [i[0] for i in similarity_scores]   # 메뉴 아이디 리스트 (메뉴 아이디는 1로 시작하므로)
    menu_cosine = [i[1] for i in similarity_scores]    # 유사도 값 저장

    for k in menu_indices:
        menu_id = menu.iloc[k]['id']
        rcm_menu.append(menu_id)

    for l in menu_cosine:
        rcm_similarity.append(l)
        
df = pd.DataFrame(zip(rcm_menu, rcm_similarity))
df.columns = ['menu_id', 'similarity_score']
df

,menu_id,similarity_score
0,59,0.535107
1,62,0.535107
2,75,0.535107
3,74,0.520231
4,61,0.501542
...,...,...
175,191,0.266190
176,245,0.221310
177,198,0.209932
178,197,0.166238


In [14]:
allergy_list = ['달걀', '우유', '밀', '콩', '땅콩', '생선', '고기', '조개', '갑각류']
user_allergy_list = list(user_allergy.iloc[0])              # 앞에 id와 user_id 포함
user_allergy_list

[64, '0', '0', '1', '0', '0', '0', '0', '0', '0', 72]

In [15]:
user_allergy_list_name = []                         # 유저별 알러지 임시로 담아둘 리스트

# 알러지에 해당하는 음식 리스트 추가
for i in range(1, len(user_allergy_list)):
    if user_allergy_list[i] == '1':
        user_allergy_list_name.append(allergy_list[i-1])
        
user_allergy_list_name

['밀']

In [32]:
# 3) 사용자 알러지가 있는 메뉴 제외

# 사용자 정보에서 알러지가 든 메뉴만 리스트업 하는 함수
def rcm_allergy(user_allergy):

    allergy_list = ['달걀', '우유', '밀', '콩', '땅콩', '생선', '고기', '조개', '갑각류']
    user_allergy_list = list(user_allergy.iloc[0])              # 앞에 id와 user_id 포함
    user_allergy_list_name = []                         # 유저별 알러지 임시로 담아둘 리스트

    # 알러지에 해당하는 음식 리스트 추가
    for i in range(2, len(user_allergy_list)):
        if user_allergy_list[i] == '1':
            user_allergy_list_name.append(allergy_list[i-2])
    
    # nutrient 파일에서 해당 알러지가 있는 메뉴 리스트업 (nutrient_menu는 메뉴 아이디 순서로 정렬되어 있음)
    allergy_menu = []

    for i in range(len(nutrient_menu)):
        allergy = str(nutrient_menu.iloc[i]['allergy'])
        allergy_info = allergy.split(',')

        # 해당 알러지가 있는 메뉴 인덱스 리스트업
        add_menu = ''
        for j in user_allergy_list_name:
            if j in allergy_info: add_menu = 1
        if add_menu == 1 and not(np.isnan(nutrient_menu.iloc[i]['menu_id'])):
            allergy_menu.append(nutrient_menu.iloc[i]['menu_id'])

    return allergy_menu

allergy_menu = rcm_allergy(user_allergy)  # 사용자 알러지가 있는 메뉴만 불러오기 (아이디 값)
rcm_menu = list(set(rcm_menu) - set(allergy_menu))

print('출력된 메뉴 길이', len(rcm_menu))
print('출력된 메뉴 리스트', rcm_menu)

출력된 메뉴 길이 105
출력된 메뉴 리스트 [11, 27, 37, 38, 40, 41, 51, 52, 57, 59, 62, 63, 64, 66, 67, 74, 75, 76, 77, 81, 83, 87, 89, 93, 98, 100, 111, 115, 116, 120, 149, 150, 151, 152, 153, 154, 155, 157, 158, 159, 160, 161, 162, 183, 184, 187, 188, 190, 191, 194, 195, 196, 197, 198, 202, 204, 214, 215, 216, 217, 224, 225, 226, 228, 230, 231, 232, 233, 234, 235, 245, 246, 247, 249, 252, 257, 259, 260, 261, 271, 283, 284, 288, 300, 301, 303, 304, 305, 306, 307, 315, 329, 330, 335, 340, 341, 342, 343, 344, 345, 360, 361, 363, 364, 365]


In [35]:
# 4) 당일에 추천된 메뉴 제외 (로그 기록 활용)

# 사용자 로그 기록을 활용하여 당일 추천된 메뉴 재추천 안되게 하는 함수
def rcm_log(user_log):
    
    today = datetime.now(timezone('Asia/Seoul'))
    today_new = today.strftime("%Y-%m-%d")  # 오늘 날짜 (2023-04-30)

    # 기존에 추천 받았던 내역 불러오기 (오늘 날짜만)
    rcm_log_list = list()
    
    for i in range(len(user_log)):
        day = user_log.loc[i]['datetime'].strftime("%Y-%m-%d")
        
        if today_new == day:
            rcm_log_list.append(user_log.loc[i]['menu_id'])
    
    return rcm_log_list

# 당일에 추천된 메뉴 아이디 리스트
rcm_log_list = rcm_log(user_log)
rcm_menu = list(set(rcm_menu) - set(rcm_log_list))
rcm_menu.sort()

print('출력된 메뉴 길이', len(rcm_menu))
print('출력된 메뉴 리스트', rcm_menu)

출력된 메뉴 길이 105
출력된 메뉴 리스트 [11, 27, 37, 38, 40, 41, 51, 52, 57, 59, 62, 63, 64, 66, 67, 74, 75, 76, 77, 81, 83, 87, 89, 93, 98, 100, 111, 115, 116, 120, 149, 150, 151, 152, 153, 154, 155, 157, 158, 159, 160, 161, 162, 183, 184, 187, 188, 190, 191, 194, 195, 196, 197, 198, 202, 204, 214, 215, 216, 217, 224, 225, 226, 228, 230, 231, 232, 233, 234, 235, 245, 246, 247, 249, 252, 257, 259, 260, 261, 271, 283, 284, 288, 300, 301, 303, 304, 305, 306, 307, 315, 329, 330, 335, 340, 341, 342, 343, 344, 345, 360, 361, 363, 364, 365]


In [36]:
# 5) 사용자가 선호하지 않은 메뉴 제외
rcm_menu = list(set(rcm_menu) - set(dislike_menu_list))
rcm_menu.sort()

print('출력된 메뉴 길이', len(rcm_menu))
print('출력된 메뉴 리스트', rcm_menu)

출력된 메뉴 길이 103
출력된 메뉴 리스트 [11, 27, 37, 38, 40, 41, 51, 52, 57, 59, 62, 63, 64, 66, 67, 74, 75, 76, 77, 81, 83, 87, 89, 93, 98, 100, 111, 115, 116, 120, 149, 150, 151, 152, 153, 154, 155, 157, 158, 159, 160, 161, 162, 184, 187, 188, 190, 191, 194, 195, 196, 197, 198, 202, 204, 214, 215, 216, 217, 224, 225, 226, 228, 230, 231, 232, 233, 234, 235, 245, 246, 247, 249, 252, 257, 260, 261, 271, 283, 284, 288, 300, 301, 303, 304, 305, 306, 307, 315, 329, 330, 335, 340, 341, 342, 343, 344, 345, 360, 361, 363, 364, 365]


In [99]:
user_price = 10000
user_weather = '맑음'
user_emotion = '기쁨'

In [100]:
# 6) 사용자 예산, 기분, 날씨 반영

# 사용자가 선택한 예산 범위의 메뉴 리스트업 하는 함수
def rcm_price(user_price):

    # 메뉴 리스트에서 해당 예산보다 작은 금액의 메뉴 리스트업
    menu_num = len(menu['id'])
    price_menu_index = []

    for i in range(menu_num):
        if menu.iloc[i]['price'] <= user_price:
            price_menu_index.append(i)
    
    return price_menu_index

# 사용자가 선택한 날씨와 감정을 반영한 메뉴를 리스트업 하는 함수
def rcm_weather_emotion(user_weather, user_emotion):

    global menu
    
    # 메뉴 리스트에서 해당 날씨, 감정에 해당하는 메뉴 리스트업
    menu_num = len(menu)
    menu_weather_list = []
    menu_emotion_list = []

    for i in range(menu_num):
        weather_list = menu.iloc[i]['weather'].split(',')
        emotion_list = menu.iloc[i]['emotion'].split(',')

        add_menu = ""
        for weather in weather_list:
            if weather == user_weather: add_menu = 1
        if add_menu == 1: menu_weather_list.append(menu.iloc[i]['id'])

        add_menu2 = ""
        for emotion in emotion_list:
            if emotion == user_emotion: add_menu2 = 1
        if add_menu2 == 1: menu_emotion_list.append(menu.iloc[i]['id'])
    
    # 사용자가 선택한 날씨, 감정이 겹치는 메뉴 리스트업
    menu_weather_emotion_list = []
    for weather_menu in menu_weather_list:
        for emotion_menu in menu_emotion_list:
            if weather_menu == emotion_menu: menu_weather_emotion_list.append(weather_menu)

    return menu_weather_emotion_list

price_menu = rcm_price(user_price)
weather_emotion_menu = rcm_weather_emotion(user_weather, user_emotion)   

price_weather_emotion = set(price_menu) & set(weather_emotion_menu)
price_weather_emotion_list = list(price_weather_emotion)

rcm_menu_new = list()
for i in rcm_menu:
    for j in price_weather_emotion_list:
        if i == j: rcm_menu_new.append(i)
            
print('출력된 메뉴 길이', len(rcm_menu_new))
print('출력된 메뉴 리스트', rcm_menu_new)

출력된 메뉴 길이 43
출력된 메뉴 리스트 [59, 62, 75, 74, 61, 63, 67, 64, 76, 116, 114, 76, 59, 62, 66, 75, 74, 211, 208, 207, 202, 206, 203, 212, 204, 116, 100, 92, 102, 149, 111, 115, 118, 117, 116, 120, 11, 257, 149, 59, 62, 66, 116]


In [111]:
# 7) 최종 추천 메뉴 리스트업
rcm_list = list(set(rcm_menu_new))
rcm_list.sort()

print('출력된 메뉴 길이', len(rcm_list))
print('출력된 메뉴 리스트', rcm_list)

출력된 메뉴 길이 31
출력된 메뉴 리스트 [11, 59, 61, 62, 63, 64, 66, 67, 74, 75, 76, 92, 100, 102, 111, 114, 115, 116, 117, 118, 120, 149, 202, 203, 204, 206, 207, 208, 211, 212, 257]


In [130]:
choice = random.randrange(0, len(rcm_list)) # 메뉴 리스트 중 한 가지 랜덤 선택 후 추천 (로그 구현 안되면 5개 보내주기)
choice_id = rcm_list[choice]                # 선택된 메뉴의 메뉴 아이디 출력
print(choice_id)

114


In [131]:
# 8) 추천된 메뉴 요약 및 음식점 정보 추가 후 전달
for i in range(len(menu)):
    if menu.iloc[i][0] == choice_id:
        menu_info = menu.iloc[i]

print(menu_info)

id                                                        114
category                                                  사이드
name                                                   달콤카라멜볼
price                                                    2200
emotion                                        배고픔,기쁨,피곤,스트레스
weather                                                 맑음,구름
image                   menu/Restaurant object (4)/달콤카라멜볼.jpg
restaurant_id                                               4
feature          사이드 달콤카라멜볼 맑음,구름 배고픔,기쁨,피곤,스트레스 우유, 밀가루, 카라멜
Name: 109, dtype: object


In [121]:
menu_id = menu_info[0]
menu_name = menu_info[2]
menu_price = menu_info[3]
restaurant_id = menu_info[7]

print('메뉴 아이디:', menu_id)
print('메뉴 이름:', menu_name)
print('메뉴 가격:', menu_price)
print('레스토랑 아이디:', restaurant_id)

for j in range(len(restaurant)):
    if restaurant.iloc[j][0] == restaurant_id:
        restaurant_info = restaurant.iloc[j]
        
restaurant_name = restaurant_info[1]
print('레스토랑 이름:', restaurant_name)

메뉴 아이디: 64
메뉴 이름: 쉬림프싸이플렉스버거
메뉴 가격: 7000
레스토랑 아이디: 4
레스토랑 이름: 맘스터치 인천대제3기숙사점


In [120]:
# 해당 메뉴의 유사도
for i in range(len(df)):
    if df.iloc[i][0] == menu_id:
        print('해당 메뉴의 유사도:', df.iloc[i][1])
        break

해당 메뉴의 유사도: 0.49980017916441544
